In [12]:
import pickle, json, re

city_data = pickle.load(open('hsinchu_city.p', 'rb'))
county_data = pickle.load(open('hsinchu_county.p', 'rb'))

In [15]:
data = city_data + county_data

In [57]:
import urllib.request
from urllib.parse import quote_plus
## use google api to get the coordinate of an address
def get_coord(address):
    google_map_url = 'http://maps.google.com/maps/api/geocode/json?address={}'
    f = urllib.request.urlopen(google_map_url.format(quote_plus(address)))
    result = json.loads(f.read().decode('utf-8'))
    coord = result['results'][0]['geometry']['location']
    return [coord['lat'], coord['lng']]

get_coord('新竹市民族路25號2-3樓')

[24.8036398, 120.9718648]

In [23]:
data[123]

{'主管機關文件單位代碼': '文961號(換)',
 '傳真號碼': '(03)5260252',
 '地址': '新竹市民族路25號2-3樓',
 '廢止、註銷文號': '',
 '廢止、註銷日期': '',
 '教室數': '4間',
 '教室面積': '129.02 平方公尺',
 '暫停招生截止日': '',
 '暫停招生文號': '',
 '暫停招生生效日': '',
 '核准科目': [{'修業期限': '3個月',
   '招生對象': '公職考試、成人',
   '核准班級數=': '2班',
   '核准科目名稱': '行政科',
   '每班核准人數': '50人',
   '每週總節(時)數': '24'},
  {'修業期限': '3個月',
   '招生對象': '公職考試、成人',
   '核准班級數=': '2班',
   '核准科目名稱': '商業科',
   '每班核准人數': '50人',
   '每週總節(時)數': '24'},
  {'修業期限': '3個月',
   '招生對象': '高中',
   '核准班級數=': '1班',
   '核准科目名稱': '國文',
   '每班核准人數': '50人',
   '每週總節(時)數': '3'},
  {'修業期限': '3個月',
   '招生對象': '高中',
   '核准班級數=': '1班',
   '核准科目名稱': '英文',
   '每班核准人數': '50人',
   '每週總節(時)數': '3'},
  {'修業期限': '3個月',
   '招生對象': '高中',
   '核准班級數=': '1班',
   '核准科目名稱': '數學',
   '每班核准人數': '50人',
   '每週總節(時)數': '3'},
  {'修業期限': '3個月',
   '招生對象': '高中',
   '核准班級數=': '1班',
   '核准科目名稱': '自然',
   '每班核准人數': '50人',
   '每週總節(時)數': '3'},
  {'修業期限': '3個月',
   '招生對象': '高中',
   '核准班級數=': '1班',
   '核准科目名稱': '社會',
   '每班核准人數': '50人',
   '每週總節(

In [90]:
items = []

fields = ['補習班名稱', '立案日期', '立案情形', '補習班類別/科目', '電話', '教室數', '教室面積', '班舍總面積', '地址']

for d in data:
    item = {}
    for f in fields:
        item[f] = d[f]

    item['科目'] = list(set([e['核准科目名稱'] for e in d['核准科目']]))
    items.append(item)

In [91]:
df_city = json.load(open('df_city.json'))
df_county = json.load(open('df_county.json'))
df_data = df_city + df_county

In [92]:
for item in items:
    item['座標'] = []
    for d in df_data:
        if d['補習班名稱'] == item['補習班名稱']:
            item['座標'] = [d['緯度'], d['經度']]


In [93]:
# '竹東鎮明德路149號' 這個地址查不到
for i, item in enumerate(items):
    if item['地址'] == '竹東鎮明德路149號2、3樓、152號1樓':
        del items[i]
        break

In [94]:
def trim_address(text):
    st = ''
    for t in text:
        if t=='號':
            st += t
            break
        else:
            st += t
    return st

# 23 個補習班沒有經緯度，用 google map api 查
for item in items:
    if len(item['座標'])==0:
        try:
            coord = get_coord(item['地址'])
            item['座標'] = coord
        except:
            coord = get_coord(trim_address(item['地址']))
            item['座標'] = coord

In [100]:
names = [r['補習班名稱'] for r in illegal_records]

In [103]:
len(names)

380

In [102]:
len(set(names))

321

In [104]:
# 加入違規紀錄

illegal_records = json.load(open('illegal_records.json'))
for item in items:
    records = []
    for r in illegal_records:
        if item['補習班名稱']==r['補習班名稱']:
            record = {}
            for field in ['日期1', '日期2', '違規']:
                record[field] = r[field]
            records.append(record)
    item['違規紀錄'] = records

In [111]:
with open('cramschool_list.json', 'w') as fp:
    json.dump(items, fp, ensure_ascii=False, indent=2)

In [112]:
# 顯示在地圖上的資料：
for_maps = []
for item in items: 
    for_maps.append([item['座標'][0], item['座標'][1], item['補習班名稱'], item['電話'], ','.join(item['科目']), len(item['違規紀錄'])>0, item['補習班類別/科目'][:-1]])

In [115]:
with open('文理補習班地圖資料.json', 'w') as fp:
    json.dump(for_maps, fp, ensure_ascii=False, indent=2)

In [149]:
for_maps

[[24.8139651,
  120.97962,
  '新竹市私立奕碩文理短期補習班',
  '0963-052559',
  '數學,美語',
  False,
  '文理類,外語類'],
 [24.783055,
  121.020526,
  '新竹市私立A1智慧教室文理短期補習班',
  '0906-861843',
  '英文,數學,英語',
  False,
  '文理類,外語類'],
 [24.7890341,
  121.0122656,
  '新竹市私立杜克美語短期補習班',
  '0975-950967',
  '美語',
  False,
  '外語類'],
 [24.8057911,
  120.9622035,
  '新竹市私立蕎立小學堂文理短期補習班',
  '(03)',
  '自然,數學,美語',
  False,
  '文理類'],
 [24.7840521, 121.020338, '新竹市私立迦南文理短期補習班', '', '國語,國文,自然', False, '文理類'],
 [24.7957431, 120.9489369, '新竹市私立青庭美語短期補習班', '0922979736', '英語', False, '外語類'],
 [24.8176931,
  120.97367,
  '新竹市私立華新文理短期補習班',
  '(03)5335001',
  '英文,數學,理化',
  False,
  '文理類'],
 [24.7944957,
  120.9775342,
  '新竹市私立華盛頓短期補習班學府分校',
  '(03)5753939',
  '英語',
  False,
  '文理類'],
 [24.8067482,
  120.9764397,
  '新竹市私立哲元文理短期補習班',
  '(03)534-0494',
  '英文,數學,國文',
  False,
  '文理類'],
 [24.798968,
  120.962752,
  '新竹市私立向日葵向陽文理短期補習班',
  '(03)5222091',
  '英文,數學,語文創作',
  False,
  '文理類'],
 [24.8123108,
  120.9838367,
  '新竹市私立魏老師文理短期補習班',
  '(03)53

In [117]:
swim_raw = '''伊藤萬-湳雅店	新竹市東區信義街68號	1		<0.3	不合格	＞5.0×102	不合格	陰性	合格	24.8030371	120.9705422
南寮溫水游泳池	新竹市海濱路240-1號	大	2	0.8	合格	＜1	合格	陰性	合格	24.8348051	120.916826
國立清華大學南大校區	新竹市東區南大路521號	訓練	9	0.4	合格	26	合格	陰性	合格	24.7945248	120.9650979
伊藤萬竹蓮校	新竹市東區竹蓮街8巷5號	10		1.3	合格	8	合格	陰性	合格	24.7973886	120.9731112
國立新竹女子高級中學	新竹市東區中華路二段270號	13		0.4	合格	30	合格	陰性	合格	24.8047755	120.9756282
私立光復高級中學	新竹市東區光復路二段153號	大	16	>1.5	不合格	＜1	合格	陰性	合格	24.7983869	120.9940564
鷹萬游泳學校-竹師融合	新竹市東區建功二路63號	18		1.5	合格	＜1	合格	陰性	合格	24.7991744	121.0032811
國立實驗中學	新竹市東區介壽路300號	淺	23	0.4	合格	＞5.0×102	不合格	陰性	合格	24.7786201	121.019041
中華大學	新竹市香山區五福路二段707號	29		0.6	合格	＜1	合格	陰性	合格	24.7569467	120.9526109'''

In [145]:
swim_maps = []
for line in swim_raw.split('\n'):
    row = line.split('\t')
    
    not_pass = '不合格' in ''.join(row[5:10])
    swim_maps.append([float(row[-2]), float(row[-1]), row[0], '', '游泳', not_pass, '體育類'])

In [150]:
swim_maps

[[24.8030371, 120.9705422, '伊藤萬-湳雅店', '', '游泳', True, '體育類'],
 [24.8348051, 120.916826, '南寮溫水游泳池', '', '游泳', False, '體育類'],
 [24.7945248, 120.9650979, '國立清華大學南大校區', '', '游泳', False, '體育類'],
 [24.7973886, 120.9731112, '伊藤萬竹蓮校', '', '游泳', False, '體育類'],
 [24.8047755, 120.9756282, '國立新竹女子高級中學', '', '游泳', False, '體育類'],
 [24.7983869, 120.9940564, '私立光復高級中學', '', '游泳', True, '體育類'],
 [24.7991744, 121.0032811, '鷹萬游泳學校-竹師融合', '', '游泳', False, '體育類'],
 [24.7786201, 121.019041, '國立實驗中學', '', '游泳', True, '體育類'],
 [24.7569467, 120.9526109, '中華大學', '', '游泳', False, '體育類']]

In [140]:
with open('游泳池地圖資料.json', 'w') as fp:
    json.dump(swim_maps, fp, ensure_ascii=False, indent=2)

In [141]:
tae_raw = '''忠誠跆拳 道新豐訓練中心	新竹縣新豐鄉松林村1鄰17號	彭庭蓁	邱瑞峰	(03)598-1004	0955-579610	24.8680266	120.9930445
忠誠跆拳道六家道館	新竹縣竹北市莊敬三路39號B1	溫益維	曾明雄	(03)658-8186	0912-243817	24.8224771	121.0249744
忠誠跆拳道館(新埔館)	新竹縣新埔鎮新關路899號	曾明雄	曾明雄	(03)588-3942	0912-243817	24.8210265	121.0959745
竹東跆拳道館	新竹縣竹東鎮中山路97號	邱雲煥	邱雲煥	(03)595-9532	0932-579920	24.7335158	121.0958771
雷霆跆拳道館	新竹縣竹東鎮東寧路一段39號2樓	古國雄	古國雄	(03)594-3120	0919-296448	24.7227923	121.0955982
忠誠跆拳道館香山分館	新竹市牛埔南路474號	楊豐吉	林煌禮	(03)540-3800	0917-260953	24.791075	120.93088
世武跆拳道武陵館	新竹市武陵路238-5號	林世育	林世育	(09)321-75319	0932-175319	24.8216754	120.9696191
忠誠跆拳道南寮館	新竹市東大路3段708巷6號	游輝平	游輝平	(09)781-21000	0978-121000	24.8407081	120.938307
維武跆拳道館	新竹市長興街103巷21號	張義勇	張義勇		0932-579065	24.7500925	120.9070525
忠誠跆拳道館學府館	新竹市學府路327號	袁知明	林煌禮	(03)575-2160	0917-260953	24.7969355	120.9843786
槶強跆拳道館香山分館	新竹市中華路五段215之1號	吳俊賢	吳俊賢		0935-329899	24.7855431	120.923191
忠誠跆拳道館光復分館	新竹市東區花園街12號6樓	彭俊憲	孫新年	(03)562-0007	0963-379699	24.804482	120.9778237
世武跆拳道館	新竹市光華二街77號	林世育	林世育	(03)542-2660	0932-175319	24.8193684	120.9756977
忠誠跆拳道館關東館	新竹市光復路一段122號	陳雋萍	孫新年	(03)579-4638	0935-509711	24.781667	121.0235695
槶強跆拳道館	新竹市竹光路100號	吳竹仁	吳竹仁		0932-110790	24.813471	120.959333
延平跆拳道訓練站	新竹市延平路一段292號	張凱鳳	吳俊賢	(03)522-5814		24.8052278	120.9519729'''

In [146]:
tae_maps = []
for line in tae_raw.split('\n'):
    row = line.split('\t')
    
    tae_maps.append([float(row[-2]), float(row[-1]), row[0], row[4],'跆拳道', False, '體育類'])

In [151]:
tae_maps

[[24.8680266,
  120.9930445,
  '忠誠跆拳 道新豐訓練中心',
  '(03)598-1004',
  '跆拳道',
  False,
  '體育類'],
 [24.8224771, 121.0249744, '忠誠跆拳道六家道館', '(03)658-8186', '跆拳道', False, '體育類'],
 [24.8210265, 121.0959745, '忠誠跆拳道館(新埔館)', '(03)588-3942', '跆拳道', False, '體育類'],
 [24.7335158, 121.0958771, '竹東跆拳道館', '(03)595-9532', '跆拳道', False, '體育類'],
 [24.7227923, 121.0955982, '雷霆跆拳道館', '(03)594-3120', '跆拳道', False, '體育類'],
 [24.791075, 120.93088, '忠誠跆拳道館香山分館', '(03)540-3800', '跆拳道', False, '體育類'],
 [24.8216754, 120.9696191, '世武跆拳道武陵館', '(09)321-75319', '跆拳道', False, '體育類'],
 [24.8407081, 120.938307, '忠誠跆拳道南寮館', '(09)781-21000', '跆拳道', False, '體育類'],
 [24.7500925, 120.9070525, '維武跆拳道館', '', '跆拳道', False, '體育類'],
 [24.7969355, 120.9843786, '忠誠跆拳道館學府館', '(03)575-2160', '跆拳道', False, '體育類'],
 [24.7855431, 120.923191, '槶強跆拳道館香山分館', '', '跆拳道', False, '體育類'],
 [24.804482, 120.9778237, '忠誠跆拳道館光復分館', '(03)562-0007', '跆拳道', False, '體育類'],
 [24.8193684, 120.9756977, '世武跆拳道館', '(03)542-2660', '跆拳道', False, '體育類'],
 [24.781

In [144]:
with open('跆拳道地圖資料.json', 'w') as fp:
    json.dump(tae_maps, fp, ensure_ascii=False, indent=2)

In [152]:
maps_data = for_maps + swim_maps + tae_maps

In [153]:
len(maps_data)

789

In [156]:
with open('全部地圖資料.json', 'w') as fp:
    json.dump(maps_data, fp, ensure_ascii=False, indent=2)

In [3]:
# comment merge
import json

sentiments = json.load(open('six_comments.json'))

In [6]:
items = json.load(open('cramschool_list.json'))

In [9]:
'我是寒'[:2]

'我是'

In [8]:
sentiments.keys()

dict_keys(['鷹萬游泳學校', '臺師補習班', '弋果美語', '忠誠跆拳道館', '伊藤萬游泳學校', '群益文理補習班'])

In [11]:
for item in items:
    for name, value in sentiments.items():
         if name[:2] in item['補習班名稱']:
                print(item['補習班名稱'])
                item['評價'] = value

弋果文化事業股份有限公司附設新竹市私立弋果美語短期補習班科園分班
弋果文化事業股份有限公司附設新竹市私立弋果美語文理短期補習班經國分班
臺師菁英文教事業股份有限公司附設新竹市私立臺師菁英1號文理短期補習班
臺師菁英文教事業股份有限公司附設新竹市私立臺師菁英6號文理短期補習班
臺師菁英文教事業股份有限公司附設新竹市私立學研文理短期補習班
臺師菁英文教事業股份有限公司附設新竹市私立臺師6號文理短期補習班
臺師菁英文教事業股份有限公司附設新竹市私立臺師7號短期補習班
臺師菁英文教事業股份有限公司附設新竹市私立臺師2號文理短期補習班
臺師菁英文教事業股份有限公司附設新竹市私立臺師5號文理短期補習班
臺師菁英文教事業股份有限公司附設新竹市私立臺師菁英文理短期補習班
臺師菁英文教事業股份有限公司附設新竹市私立臺師文理短期補習班
新竹市弋果美語短期補習班
弋果文化事業股份有限公司附設新竹市私立旗艦美語短期補習班
私立弋果美語短期補習班
臺師菁英文教事業股份有限公司附設新竹縣私立臺師9號文理短期補習班
臺師菁英文教事業股份有限公司附設新竹縣私立臺師1號文理短期補習班


In [12]:
for item in items:
    if '評價' in item:
        print(item)

{'立案情形': '已立案', '班舍總面積': '394.94平方公尺', '電話': '(03)5630136', '補習班類別/科目': '外語類,', '地址': '新竹市東區金山街2號3樓', '評價': {'rate': [179, 265, 79], 'comments': [{'score': 0.10000000149011612, 'title': '[台北] 松山敦化弋果美語徵行政櫃檯- 看板job - 批踢踢實業坊', 'summary': '請各位資方配合遵守。弋果美語行政櫃檯1.服務家長及學員(接聽電話,問題解決) 2.介紹班內產品課程、受理櫃檯報名手續 3.協助招生及活動辦理', 'url': 'https://www.ptt.cc/bbs/job/M.1478369666.A.4F0.html'}, {'score': 0.10000000149011612, 'title': '[台北] 松山敦化弋果美語徵行政櫃檯- 看板job - 批踢踢實業坊', 'summary': '請各位資方配合遵守。弋果美語行政櫃檯1.服務家長及學員(接聽電話,問題解決) 2.介紹班內產品課程、受理櫃檯報名手續 3.協助招生及活動辦理', 'url': 'https://www.ptt.cc/bbs/job/M.1479310045.A.9C6.html'}, {'score': 0.10000000149011612, 'title': '[台北] 弋果美語松山敦化分校徵中師行政人員- 看板job - 批踢踢實業坊', 'summary': '請各位資方配合遵守。弋果美語松山敦化分校中師行政人員1.服務家長及學員 2.課堂班級管理 3.英語教學輔導', 'url': 'https://www.ptt.cc/bbs/job/M.1504142958.A.F94.html'}, {'score': 0.30000001192092896, 'title': '[北部] 弋果美語蘆竹光明分校誠徵中師- 看板job - 批踢踢實業坊', 'summary': '請各位資方配合遵守。大學畢業以上，英語具中等溝通與書寫能力，可以與外師配合教學為主。月薪未達21009元 會被刪文 時薪工作請貼Part-Time板王小姐', 'url': 'https://www.p

In [13]:
len(items)

764